In [1]:
import csv
from datetime import datetime as dt
from tabulate import tabulate

In [2]:
filename = 'expense_file.csv'
file_headers = ['date', 'category', 'amount', 'description', 'budget']

has_header = False

try:

    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.reader(csvfile)
        
        # Read the first row (header)
        headers = next(reader, None)
        if headers is not None:
            has_header = True
    
    if not has_header:
        with open(filename, mode='w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=file_headers)
            
            writer.writeheader() 
            
except FileNotFoundError:
    with open(filename, mode='w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=file_headers)
            
            writer.writeheader()
    

### Save Expense

In [4]:
def save_expense(dict_list):
    filename = 'expense_file.csv'
    with open(filename, mode='a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=dict_list[0].keys())
        
        if csvfile.tell() == 0:
            writer.writeheader()
        
        writer.writerows(dict_list)

In [5]:
def read_integer():
    while True:
        cat_choice = input()
        try:
            cat_choice = int(cat_choice)
            break
        except Exception:
            print("Enter a valid integer")
    return cat_choice


def read_float():
    while True:
        cat_choice = input()
        try:
            cat_choice = float(cat_choice)
            break
        except Exception:
            print("Enter a valid Float")
    return cat_choice

### Add Expense

In [7]:
def add_expense():
    print("Hi, Enter the below details to add expenses")
    date_format = "%Y-%m-%d"
    
    while True:
        date_input = input("Enter date in YYYY-MM-DD format:")
        try:
            parsed_dt = dt.strptime(date_input, date_format)
            break
        except Exception:
            print("Invalid Date Format")

    print("Select the Expense category \n 1 -> Food \n 2 -> Travel \n 3 -> Clothing \n 4 -> Entertainment \n 5 -> Misc")

    while True:
        cat_choice = input()
        try:
            cat_choice = int(cat_choice)
            break
        except Exception:
            print("Enter a valid integer")
    
    if cat_choice == 1:
        exp_cat = 'Food'
    elif cat_choice == 2:
        exp_cat = 'Travel'
    elif cat_choice == 3:
        exp_cat = 'Clothing'
    elif cat_choice == 4:
        exp_cat = 'Entertainment'
    elif cat_choice == 5:
        exp_cat = 'Misc'
    else:
        cat_choice = 'Others'

    print("Enter expense amount")
    amt = read_float()

    exp_desc = input("Enter the expense description")

    # Storing in a Dict
    dict_list = []
    row = {
        "date" : parsed_dt,
        "category" : exp_cat,
        "amount" : amt,
        "description" : exp_desc,
        "budget":100   # Default expense
    }
    dict_list.append(row)

    return dict_list   


### View Expense

In [9]:
def view_expense():
    filename = 'expense_file.csv'
    i = 0
    with open(filename, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            i+=1
        if i == 0:
             print("No available expenses to show. Add expense to view")
            
    with open(filename, mode='r', newline='') as file:
        reader = csv.DictReader(file)              
        for row in reader:
            if any(value == '' for value in row.values()):
                print(f"Row contains null values: {row}")
    
    with open(filename, mode='r', newline='') as file:
        reader = csv.DictReader(file)             
        data = [row for row in reader]
        print(tabulate(data, headers="keys", tablefmt="grid"))

### save budget

In [11]:
def save_budget(file_path, month, update_field, new_value):
    rows = []
    
    # Read the existing CSV file
    with open(file_path, mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        rows = list(reader)  # Load all rows into memory

    # Update the specified rows
    for row in rows:
        # Convert the date string to a datetime object
        date_str = row['date']
        date_obj = dt.strptime(date_str, '%Y-%m-%d %H:%M:%S')  # Adjust format as necessary

        # Check if the month matches
        if date_obj.month == month:
            row[update_field] = new_value  # Update the specified field with new value

    # Write the updated rows back to the CSV file
    with open(file_path, mode='w', newline='') as csvfile:
        fieldnames = rows[0].keys()  # Get field names from the first row
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()  # Write header
        writer.writerows(rows)  # Write updated rows

### Set Budget

In [13]:
def set_budget():
    print("Enter the Budget Month Number")
    month = read_integer()

    print("Enter the Budget for the month")
    month_budget = read_float()

    save_budget(filename, month, 'budget', month_budget)

    filtered_rows = []
    
    with open(filename , mode='r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            # Convert the date string to a datetime object
            date_str = row['date']
            date_obj = dt.strptime(date_str, '%Y-%m-%d %H:%M:%S')
            
            # Check if the month matches
            if date_obj.month == month:
                row['budget'] = month_budget
                filtered_rows.append(row)
    for row in filtered_rows:
        try:
            row['amount'] = float(row['amount'])
        except ValueError:
            row['amount'] = None  # or handle it in another way


    if ((sum(row['amount'] for row in filtered_rows)) > (sum(row['budget'] for row in filtered_rows)/len(filtered_rows))):
        print("Warning! Crossed budget by",((sum(row['amount'] for row in filtered_rows)) - (sum(row['budget'] for row in filtered_rows)/len(filtered_rows))))
    else:
        print("Amount left is",((sum(row['amount'] for row in filtered_rows)/len(filtered_rows)) - (sum(row['budget'] for row in filtered_rows))))
            
    
    
        

## Driver Funtion

In [15]:
add_count = 0
dict_list_exp = []
while True:
    print('Enter your choice.....')
    print('1 -> Add expense')
    print('2 -> View expense')
    print('3 -> Track budget')
    print('4 -> Save expense to a file')
    print('5 -> Exit')
    
    choice = read_integer()
    if choice == 1:
        dict_list = add_expense()
        add_count += 1
        dict_list_exp = dict_list_exp + dict_list
    elif choice == 2:
        view_expense()
    elif choice == 3:
        set_budget()
    elif choice == 4:
        save_expense(dict_list_exp)
    elif choice == 5:
        break
    else:
        print("enter a valid number")

Enter your choice.....
1 -> Add expense
2 -> View expense
3 -> Track budget
4 -> Save expense to a file
5 -> Exit


 1


Hi, Enter the below details to add expenses


Enter date in YYYY-MM-DD format: 12-11-2024


Invalid Date Format


...

 400


Warning! Crossed budget by 72.0
Enter your choice.....
1 -> Add expense
2 -> View expense
3 -> Track budget
4 -> Save expense to a file
5 -> Exit


 2


+---------------------+------------+----------+---------------+----------+
| date                | category   |   amount | description   |   budget |
+=====================+============+==========+===============+==========+
| 1111-11-11 00:00:00 | Food       |      111 | aadcd         |      400 |
+---------------------+------------+----------+---------------+----------+
| 1111-11-11 00:00:00 | Food       |      111 | aadcd         |      400 |
+---------------------+------------+----------+---------------+----------+
| 2024-11-11 00:00:00 | Food       |      100 | lunch         |      400 |
+---------------------+------------+----------+---------------+----------+
| 2024-11-10 00:00:00 | Travel     |      150 | To Office     |      400 |
+---------------------+------------+----------+---------------+----------+
Enter your choice.....
1 -> Add expense
2 -> View expense
3 -> Track budget
4 -> Save expense to a file
5 -> Exit


 5
